# Telco Customer Churn for ICP4D

We'll use this notebook to create a machine learning model to predict customer churn.

# 1.0 Install required packages

In [ ]:
!pip install --user watson-machine-learning-client --upgrade | tail -n 1

# 2.0 Load and Clean data
We'll load our data as a pandas data frame.

* Highlight the cell below by clicking it.
* Click the `10/01` "Find data" icon in the upper right of the notebook.
* If you are using the [ICP4D Learning Path] and have the [Virtualized data], begin by choosing the `Remote` tab. Then choose your virtualized data (i.e. User<xyz>.billingProductCustomers), click `Insert to code` and choose `Insert Pandas DataFrame`.
* If you are using this code pattern outside of the [ICP4D Learning Path], add the locally uploaded file `Telco-Customer-Churn.csv` by choosing the 'Local` tab. Then choose the `Telco-Customer-Churn.csv`, click `Insert to code` and choose `Insert Pandas DataFrame`.
* The code to bring the data into the notebook environment and create a Pandas DataFrame will be added to the cell below.
* Run the cell


In [ ]:
# Place cursor below and insert the Pandas DataFrame for the Telco churn data


We'll use the Pandas naming convention `df` for our DataFrame.  Make sure that the cell below uses the name for the dataframe used above, i.e df1, df2,... dfX for the remote data using Data Virtualization, or df_data1, df_data2, ... if using local data.

In [ ]:
# df = df1
df = df_data_1

### 2.1 Drop CustomerID feature (column)

In [ ]:
df = df.drop('customerID', axis=1)
df.head(5)

### 2.2 Examine the data types of the features

In [ ]:
df.info()

### 2.3 Convert TotalCharges column to numeric as it is detected as object

In [ ]:
new_col = pd.to_numeric(df.iloc[:, 18], errors='coerce')
new_col

### 2.4 Modify our dataframe to reflect the new datatype

In [ ]:
df.iloc[:, 18] = pd.Series(new_col)
df



### 2.5 Any NaN values should be removed to create a more accurate model. Prior examination shows NaN values for `TotalCharges`

In [ ]:
# Check if we have any NaN values
df.isnull().values.any()

Set `nan_column` to the column number for TotalCharges (starting at 0).

In [ ]:
nan_column = df.columns.get_loc("TotalCharges")
print(nan_column)

In [ ]:
# Handle missing values for nan_column (TotalCharges)

from sklearn.preprocessing import Imputer

imp = Imputer(missing_values="NaN", strategy="mean")

df.iloc[:, nan_column] = imp.fit_transform(df.iloc[:, nan_column].values.reshape(-1, 1))
df.iloc[:, nan_column] = pd.Series(df.iloc[:, nan_column])

In [ ]:
# Check if we have any NaN values
df.isnull().values.any()

### 2.6 Visualize data

In [ ]:
import json
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, StandardScaler
import sklearn.feature_selection
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn import metrics

In [ ]:
# Plot Tenure Frequency count
sns.set(style="darkgrid")
sns.set_palette("hls", 3)
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.countplot(x="tenure", hue="Churn", data=df)

In [ ]:
# Plot Tenure Frequency count
sns.set(style="darkgrid")
sns.set_palette("hls", 3)
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.countplot(x="Contract", hue="Churn", data=df)

In [ ]:
# Plot Tenure Frequency count
sns.set(style="darkgrid")
sns.set_palette("hls", 3)
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.countplot(x="TechSupport", hue="Churn", data=df)


In [ ]:
# Create Grid for pairwise relationships
gr = sns.PairGrid(df, size=5, hue="Churn")
gr = gr.map_diag(plt.hist)
gr = gr.map_offdiag(plt.scatter)
gr = gr.add_legend()

In [ ]:
totalCharge  = df.columns.get_loc("TotalCharges")
print(nan_column)

In [ ]:
# Set up plot size
fig, ax = plt.subplots(figsize=(6,6))

# Attributes destribution
a = sns.boxplot(orient="v", palette="hls", data=df.iloc[:, totalCharge], fliersize=14)


In [ ]:
# Total Charges data distribution
histogram = sns.distplot(df.iloc[:, totalCharge], hist=True)
plt.show()

In [ ]:
tenure  = df.columns.get_loc("tenure")
print(tenure)

In [ ]:
# Tenure data distribution
histogram = sns.distplot(df.iloc[:, tenure], hist=True)
plt.show()

In [ ]:

monthly = df.columns.get_loc("MonthlyCharges")
print(monthly)

In [ ]:
# Monthly Charges data distribution
histogram = sns.distplot(df.iloc[:, monthly], hist=True)
plt.show()


Understand Data Distribution¶


# 3.0 Create a model

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.createDataFrame(df)
df_data.head()

### 3.1 Split the data into training and test sets

In [ ]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

### 3.2 Examine the Spark DataFrame Schema
Look at the data types to determine requirements for feature engineering

In [ ]:
spark_df.printSchema()

### 3.3 Use StringIndexer to encode a string column of labels to a column of label indices

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model


si_gender = StringIndexer(inputCol = 'gender', outputCol = 'gender_IX')
si_Partner = StringIndexer(inputCol = 'Partner', outputCol = 'Partner_IX')
si_Dependents = StringIndexer(inputCol = 'Dependents', outputCol = 'Dependents_IX')
si_PhoneService = StringIndexer(inputCol = 'PhoneService', outputCol = 'PhoneService_IX')
si_MultipleLines = StringIndexer(inputCol = 'MultipleLines', outputCol = 'MultipleLines_IX')
si_InternetService = StringIndexer(inputCol = 'InternetService', outputCol = 'InternetService_IX')
si_OnlineSecurity = StringIndexer(inputCol = 'OnlineSecurity', outputCol = 'OnlineSecurity_IX')
si_OnlineBackup = StringIndexer(inputCol = 'OnlineBackup', outputCol = 'OnlineBackup_IX')
si_DeviceProtection = StringIndexer(inputCol = 'DeviceProtection', outputCol = 'DeviceProtection_IX')
si_TechSupport = StringIndexer(inputCol = 'TechSupport', outputCol = 'TechSupport_IX')
si_StreamingTV = StringIndexer(inputCol = 'StreamingTV', outputCol = 'StreamingTV_IX')
si_StreamingMovies = StringIndexer(inputCol = 'StreamingMovies', outputCol = 'StreamingMovies_IX')
si_Contract = StringIndexer(inputCol = 'Contract', outputCol = 'Contract_IX')
si_PaperlessBilling = StringIndexer(inputCol = 'PaperlessBilling', outputCol = 'PaperlessBilling_IX')
si_PaymentMethod = StringIndexer(inputCol = 'PaymentMethod', outputCol = 'PaymentMethod_IX')


In [ ]:
si_Label = StringIndexer(inputCol="Churn", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

### 3.4 Create a single vector

In [ ]:
va_features = VectorAssembler(inputCols=['gender_IX',  'SeniorCitizen', 'Partner_IX', 'Dependents_IX', 'PhoneService_IX', 'MultipleLines_IX', 'InternetService_IX', \
                                         'OnlineSecurity_IX', 'OnlineBackup_IX', 'DeviceProtection_IX', 'TechSupport_IX', 'StreamingTV_IX', 'StreamingMovies_IX', \
                                         'Contract_IX', 'PaperlessBilling_IX', 'PaymentMethod_IX', 'TotalCharges', 'MonthlyCharges'], outputCol="features")

### 3.5 Create a pipeline, and fit a model using RandomForestClassifier 
Assemble all the stages into a pipeline. We don't expect a clean linear regression, so we'll use RandomForestClassifier to find the best decision tree for the data.

In [ ]:
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[si_gender, si_Partner, si_Dependents, si_PhoneService, si_MultipleLines, si_InternetService, si_OnlineSecurity, si_OnlineBackup, si_DeviceProtection, \
                            si_TechSupport, si_StreamingTV, si_StreamingMovies, si_Contract, si_PaperlessBilling, si_PaymentMethod, si_Label, va_features, \
                            classifier, label_converter])

model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

# 4.0 Save the model and test data

Add a unique name for MODEL_NAME.

In [ ]:
MODEL_NAME = "myname model"

### 4.1 Save the model to ICP4D local Watson Machine Learning

In [ ]:
from dsx_ml.ml import save

save(name=MODEL_NAME, model=model, test_data=test_data, algorithm_type='Classification',
     description='This is a SparkML Model to Classify Telco Customer Churn Risk')

### 4.2 Write the test data without label to a .csv so that we can later use it for batch scoring

In [ ]:
write_score_CSV=test_data.toPandas().drop(['Churn'], axis=1)
write_score_CSV.to_csv('../datasets/TelcoCustomerSparkMLBatchScore.csv', sep=',', index=False)

### 4.3 Write the test data to a .csv so that we can later use it for evaluation

In [ ]:
write_eval_CSV=test_data.toPandas()
write_eval_CSV.to_csv('../datasets/TelcoCustomerSparkMLEval.csv', sep=',', index=False)

## Congratulations, you have created a model based on customer churn data, and deployed it to Watson Machine Learning!